## HMC preprocessing

### Convert from hg38 to hg19

## Method #1: liftover

In [ ]:
python $PRS_TOOLS/liftOverPlink/liftOverPlink.py --map $PRS_DATASETS/humc/original/raw/liftover/ds38.map --out $PRS_DATASETS/hadassa/original/raw/liftover/lifted --chain $PRS_TOOLS/liftOverPlink/hg38ToHg19.over.chain
python $PRS_TOOLS/liftOverPlink/rmBadLifts.py --map lifted.map --out good_lifted.map --log bad_lifted.dat
cut -f 2 bad_lifted.dat > to_exclude.dat
cut -f 4 lifted.bed.unlifted | sed "/^#/d" >> to_exclude.dat
plink --file ds38 --recode --out lifted_filtered --exclude to_exclude.dat --allow-extra-chr
plink --ped lifted_filtered.ped --map good_lifted.map --recode --out final


## Method #2: convert vcf to hg19 using annotations included in the vcf file

#### copy headers

In [ ]:
# Retrieve first data lines
a=$(awk '{if(substr($0,1,1)!="#"){print NR; exit;}}' humc19.vcf)
# Copy headers
head -n $(($a-1)) humc19.vcf > humc19_reformatted.vcf

####  reformat data

In [ ]:
tail -n +66  humc19.vcf  | awk '{len=split($8,ar,";"); is_hg19=0; for(i=1;i<=len; i++){if (ar[i] ~ /hg19_chr/){split(ar[i],a,"="); a_val=a[2]}; if (ar[i] ~ /hg19_position/){split(ar[i],b,"="); b_val=b[2]; is_hg19=1}}; if(is_hg19==1){printf a_val"\t"b_val"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9; for(i=1; i<=181; i++){printf "\t"$(9+i)}; printf "\n"} }' >> humc19_reformatted.vcf

#### Convert to bed files

In [ ]:
plink --vcf humc19_reformatted.vcf --vcf-idspace-to $ --double-id --make-bed --allow-extra-chr --out ../ds